In [35]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.externals import joblib
pd.set_option('display.max_columns', None)  

In [36]:
df = pd.read_csv('../sample_csv_data/ohport_downsampling_iot_flow.csv')
df = df.drop(columns=['Unnamed: 0'])

In [37]:
df.head()

,timeStampFirst,timeStampLast,duration,linkProtocol,networkProtocol,transportProtocol,applicationProtocol,srcMac,dstMac,srcIP,dstIP,srcPort,dstPort,packetCount,totalPayloadByte,appplicationPayloadByte,iot,device,dns,dhcp-server,http,ntp,https,smtp-ssl,imap-ssl,ssdp,icslap,stm-pproc,stun,ws-discovery,upnp-evnt,xmpp,android,dstPort25050,dstPort49152,dstPort49153,dstPort49154,wellKnownDstPort,ephemeralDstPort,registedDstPort,srcPort49152,srcPort49153
0,1475671755714,1475671756741,1027,Ethernet,IPv4,TCP,NaN,ec:1a:59:83:28:11,ec:1a:59:79:f4:89,192.168.1.193,192.168.1.223,49153.0,3538.0,6,597,0,True,Belkin wemo motion sensor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1475020226003,1475020226003,0,Ethernet,IPv4,UDP,DNS,00:16:6c:ab:6b:88,14:cc:20:51:33:ea,192.168.1.249,192.168.1.1,50591.0,53.0,2,148,0,True,Samsung SmartCam,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1476208177398,1476208177427,29,Ethernet,IPv4,TCP,NaN,ec:1a:59:79:f4:89,ec:1a:59:83:28:11,192.168.1.223,192.168.1.193,49152.0,4018.0,7,4087,0,True,Belkin Wemo switch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,1474599250064,1474599250086,22,Ethernet,IPv4,TCP,NaN,00:16:6c:ab:6b:88,ec:1a:59:83:28:11,192.168.1.249,192.168.1.193,49152.0,4415.0,5,1508,0,True,Samsung SmartCam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1476049394301,1476049394604,303,Ethernet,IPv4,TCP,NaN,44:65:0d:56:cc:d3,14:cc:20:51:33:ea,192.168.1.240,93.184.216.34,53974.0,80.0,4,272,0,True,Amazon Echo,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [38]:
# one hot for network & transport protocol
one_hot = pd.get_dummies(df['networkProtocol'])
df = df.join(one_hot)
one_hot = pd.get_dummies(df['transportProtocol'])
df = df.join(one_hot)

# device name to number
device_list = df.device.unique()
print(device_list)
device_dict = dict(zip(device_list, range(len(device_list))))
mdf = df.replace({'device': device_dict})

['Belkin wemo motion sensor' 'Samsung SmartCam' 'Belkin Wemo switch'
 'Amazon Echo' 'Insteon Camera' 'PIX-STAR Photo-frame' 'Smart Things'
 'Withings Smart Baby Monitor' 'Netatmo Welcome'
 'Light Bulbs LiFX Smart Bulb' 'Withings Aura smart sleep sensor'
 'TP-Link Day Night Cloud camera' 'Netatmo weather station' 'HP Printer'
 'Triby Speaker' 'TP-Link Smart plug' 'iHome' 'Dropcam'
 'Withings Smart scale' 'NEST Protect smoke alarm' 'Nest Dropcam'
 'Blipcare Blood Pressure meter' 'Samsung Galaxy Tab' 'Laptop'
 'Android Phone' 'IPhone' 'MacBook' 'MacBook/Iphone']


In [39]:
mdf.head()

,timeStampFirst,timeStampLast,duration,linkProtocol,networkProtocol,transportProtocol,applicationProtocol,srcMac,dstMac,srcIP,dstIP,srcPort,dstPort,packetCount,totalPayloadByte,appplicationPayloadByte,iot,device,dns,dhcp-server,http,ntp,https,smtp-ssl,imap-ssl,ssdp,icslap,stm-pproc,stun,ws-discovery,upnp-evnt,xmpp,android,dstPort25050,dstPort49152,dstPort49153,dstPort49154,wellKnownDstPort,ephemeralDstPort,registedDstPort,srcPort49152,srcPort49153,IPv4,IPv6,TCP,UDP
0,1475671755714,1475671756741,1027,Ethernet,IPv4,TCP,NaN,ec:1a:59:83:28:11,ec:1a:59:79:f4:89,192.168.1.193,192.168.1.223,49153.0,3538.0,6,597,0,True,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1,0,1,0
1,1475020226003,1475020226003,0,Ethernet,IPv4,UDP,DNS,00:16:6c:ab:6b:88,14:cc:20:51:33:ea,192.168.1.249,192.168.1.1,50591.0,53.0,2,148,0,True,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0,0,1
2,1476208177398,1476208177427,29,Ethernet,IPv4,TCP,NaN,ec:1a:59:79:f4:89,ec:1a:59:83:28:11,192.168.1.223,192.168.1.193,49152.0,4018.0,7,4087,0,True,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0,1,0
3,1474599250064,1474599250086,22,Ethernet,IPv4,TCP,NaN,00:16:6c:ab:6b:88,ec:1a:59:83:28:11,192.168.1.249,192.168.1.193,49152.0,4415.0,5,1508,0,True,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0,1,0
4,1476049394301,1476049394604,303,Ethernet,IPv4,TCP,NaN,44:65:0d:56:cc:d3,14:cc:20:51:33:ea,192.168.1.240,93.184.216.34,53974.0,80.0,4,272,0,True,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0,1,0


In [14]:
features = [
    'http',
    'ntp',
    'https',
    'smtp-ssl',
    'ws-discovery',
    'upnp-evnt',
    'srcPort49152',
    'srcPort49153',
    'ephemeralDstPort', 
    'registedDstPort',
    'IPv4',
    'IPv6',
    'TCP',
    'UDP',
    'duration',
    'packetCount',
    'totalPayloadByte',
    'appplicationPayloadByte'
]
print(len(features))
X = df[features].values[:,:]
Y = df[['iot']].values[:]
Y = Y.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 100)

18


In [11]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("accurency is", accuracy_score(y_test,y_pred), "\nf1 score is", f1_score(y_test, y_pred), "\n")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


accurency is 0.986890828986134 
f1 score is 0.9868531620178362 



In [12]:
i = 0
for im in clf.feature_importances_:
    print(str(im*100), features[i])
    i += 1

2.4049578456159653 http
2.2642547692523967 ntp
10.679770272435967 https
1.4153105410025757 smtp-ssl
1.1366183558446141 ws-discovery
2.636096184039944 upnp-evnt
3.3219039959379173 srcPort49152
3.0133330365933126 srcPort49153
2.2497790965977984 ephemeralDstPort
3.237642021148853 registedDstPort
1.275407334272389 IPv4
1.0931478801903156 IPv6
1.2320648072274778 TCP
1.2514810800933633 UDP
23.382243095601698 duration
9.224148209561287 packetCount
21.698891043528494 totalPayloadByte
8.482950431055635 appplicationPayloadByte


In [13]:
joblib.dump(clf, 'saved_model_minimized.pkl')

['saved_model_minimized.pkl']

In [15]:
# multiclass

In [43]:
features = [
    'http',
    'ntp',
    'https',
    'smtp-ssl',
    'ws-discovery',
    'upnp-evnt',
    'srcPort49152',
    'srcPort49153',
    'ephemeralDstPort', 
    'registedDstPort',
    'IPv4',
    'IPv6',
    'TCP',
    'UDP',
    'duration',
    'packetCount',
    'totalPayloadByte',
    'appplicationPayloadByte'
]
print(len(features))
X = mdf[features].values[:,:]
Y = mdf[['device']].values[:]
Y = Y.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 100)

18


In [44]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [49]:
print("accurency is", accuracy_score(y_test,y_pred), "\nf1 score is", f1_score(y_test, y_pred, average=None), "\n")
scores = f1_score(y_test, y_pred, average=None)

accurency is 0.9178311755007162 
f1 score is [0.98703888 0.98782494 0.98518621 0.98608427 0.97255871 0.71428571
 0.88135593 0.8923476  0.91578947 0.96700796 0.88017429 0.93586698
 0.90196078 0.84955752 0.58441558 0.84615385 0.71794872 0.
 0.66666667 0.58823529 0.5        0.85718757 0.92669934 0.4470405
 0.20689655 0.73831089 0.3902439 ] 



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [47]:
device_list

array(['Belkin wemo motion sensor', 'Samsung SmartCam',
       'Belkin Wemo switch', 'Amazon Echo', 'Insteon Camera',
       'PIX-STAR Photo-frame', 'Smart Things',
       'Withings Smart Baby Monitor', 'Netatmo Welcome',
       'Light Bulbs LiFX Smart Bulb', 'Withings Aura smart sleep sensor',
       'TP-Link Day Night Cloud camera', 'Netatmo weather station',
       'HP Printer', 'Triby Speaker', 'TP-Link Smart plug', 'iHome',
       'Dropcam', 'Withings Smart scale', 'NEST Protect smoke alarm',
       'Nest Dropcam', 'Blipcare Blood Pressure meter',
       'Samsung Galaxy Tab', 'Laptop', 'Android Phone', 'IPhone',
       'MacBook', 'MacBook/Iphone'], dtype=object)

In [51]:
for i, j in zip(scores, device_list):
    print(i,j)

0.98703888334995 Belkin wemo motion sensor
0.9878249424152682 Samsung SmartCam
0.9851862107157691 Belkin Wemo switch
0.9860842674913027 Amazon Echo
0.9725587144622991 Insteon Camera
0.7142857142857144 PIX-STAR Photo-frame
0.8813559322033899 Smart Things
0.8923476005188068 Withings Smart Baby Monitor
0.9157894736842105 Netatmo Welcome
0.9670079635949943 Light Bulbs LiFX Smart Bulb
0.8801742919389978 Withings Aura smart sleep sensor
0.9358669833729215 TP-Link Day Night Cloud camera
0.9019607843137254 Netatmo weather station
0.8495575221238938 HP Printer
0.5844155844155844 Triby Speaker
0.8461538461538461 TP-Link Smart plug
0.717948717948718 iHome
0.0 Dropcam
0.6666666666666666 Withings Smart scale
0.5882352941176471 NEST Protect smoke alarm
0.5 Nest Dropcam
0.857187573362548 Blipcare Blood Pressure meter
0.9266993394417218 Samsung Galaxy Tab
0.44704049844236765 Laptop
0.20689655172413793 Android Phone
0.738310893512852 IPhone
0.3902439024390244 MacBook


In [52]:
df.groupby('device').size()

device
Amazon Echo                          8766
Android Phone                        2580
Belkin Wemo switch                  12251
Belkin wemo motion sensor           16833
Blipcare Blood Pressure meter           2
Dropcam                                13
HP Printer                            357
IPhone                                 63
Insteon Camera                       6580
Laptop                              31019
Light Bulbs LiFX Smart Bulb          1367
MacBook                              6932
MacBook/Iphone                        294
NEST Protect smoke alarm               23
Nest Dropcam                           10
Netatmo Welcome                      1009
Netatmo weather station               650
PIX-STAR Photo-frame                  477
Samsung Galaxy Tab                  20773
Samsung SmartCam                    10035
Smart Things                          181
TP-Link Day Night Cloud camera        644
TP-Link Smart plug                    100
Triby Speaker              